In [1]:
print("⏳ Setting up environment for HMTAS inference...")

# --- Step 1: Ensure clean base ---
!pip uninstall -y bleurt tensorflow tensorflow-text -q || true
!rm -rf /root/.cache/pip /root/.cache/huggingface /root/.cache/nltk

# --- Step 2: Install core stable packages (quietly) ---
!pip install -q -U numpy==1.26.4 scikit-learn==1.3.2
!pip install -q transformers==4.41.2 datasets==2.18.0 sentence-transformers==2.7.0
!pip install -q rouge-score==0.1.2 bert-score==0.3.13
!pip install -q networkx==3.2.1 hdbscan==0.8.33 umap-learn==0.5.5
!pip install -q matplotlib seaborn nltk

# --- Step 3: Torch (CPU) install (GPU uses default runtime CUDA) ---
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

# --- Step 4: Download NLTK resources silently ---
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

print("✅ Environment ready for HMTAS inference and visualization.")


⏳ Setting up environment for HMTAS inference...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 104.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.3.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [2]:
# hmtas_flan_t5_fixed.py
import warnings, os, time, random, numpy as np, pandas as pd, torch, torch.nn.functional as F
from pathlib import Path
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer
from bert_score import score as bert_score_calculator
from nltk.tokenize import sent_tokenize
import hdbscan, networkx as nx

warnings.filterwarnings("ignore")
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# ---------------- CONFIG ----------------
class Config:
    total_test_samples = 100
    backbone = "google/flan-t5-base"
    max_input_length = 512
    max_target_length = 180
    token_budget = 380        # dynamically capped later
    min_cluster_size = 3
    per_cluster_topk = 3
    fallback_topk = 8
    sim_threshold = 0.15
    pagerank_alpha = 0.85
    num_beams = 6
    length_penalty = 1.3
    min_length = 40
    repetition_penalty = 1.1
    no_repeat_ngram_size = 3
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 42
    output_dir = Path("./hmtas_output")

config = Config()

# ---------------- UTILS ----------------
def set_seed(seed):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def mkdir(p): p.mkdir(parents=True, exist_ok=True)

# ---------------- ENCODER + CLUSTER ----------------
class SentenceEncoderGPU:
    def __init__(self, device):
        self.model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
    def encode(self, sents):
        if not sents:
            return torch.empty(0, 384, device=config.device)
        return self.model.encode(sents, convert_to_tensor=True, device=config.device, show_progress_bar=False)
    def cosine(self,a,b):
        a,b=F.normalize(a,2),F.normalize(b,2)
        return torch.mm(a,b.t())

class HMTASClusterer:
    def __init__(self,cfg): self.cfg=cfg
    def cluster(self,emb):
        e=emb.detach().cpu().numpy()
        if e.shape[0]<self.cfg.min_cluster_size:
            return np.zeros(e.shape[0],dtype=int)
        c=hdbscan.HDBSCAN(min_cluster_size=self.cfg.min_cluster_size,min_samples=1)
        labels=c.fit_predict(e)
        if (labels==-1).any():
            non=np.where(labels!=-1)[0]
            for i in np.where(labels==-1)[0]:
                sims=F.cosine_similarity(torch.tensor(e[i]).unsqueeze(0),torch.tensor(e[non]),dim=-1)
                labels[i]=int(labels[non][sims.argmax().item()])
        return labels

# ---------------- PROCESSOR ----------------
class HMTASProcessor:
    def __init__(self,cfg,tok):
        self.cfg=cfg; self.tok=tok
        self.encoder=SentenceEncoderGPU(cfg.device)
        self.clusterer=HMTASClusterer(cfg)
    def preprocess(self,docs):
        sents=[]
        for d in docs:
            sents.extend([s for s in sent_tokenize(d) if 8<len(s.split())<100])
        return sents
    def create_guided_input(self,docs):
        sents=self.preprocess(docs)
        if not sents:
            return "Summarize: "+" ".join(docs)
        emb=self.encoder.encode(sents)
        sim=self.encoder.cosine(emb,emb)
        labels=self.clusterer.cluster(emb)
        chosen=[]
        for l in np.unique(labels):
            idxs=np.where(labels==l)[0].tolist()
            sub=sim[idxs][:,idxs].cpu().numpy()
            sub[sub<self.cfg.sim_threshold]=0
            np.fill_diagonal(sub,0)
            G=nx.from_numpy_array(sub)
            pr=nx.pagerank(G,alpha=self.cfg.pagerank_alpha)
            ranked=sorted(pr.items(),key=lambda x:x[1],reverse=True)
            chosen.extend([idxs[i] for i,_ in ranked[:self.cfg.per_cluster_topk]])
        chosen=list(dict.fromkeys(chosen))
        if len(chosen)<self.cfg.fallback_topk:
            sums=sim.sum(dim=1).cpu().numpy()
            topk=np.argsort(-sums)[:self.cfg.fallback_topk]
            chosen=list(dict.fromkeys(chosen+topk.tolist()))
        total,final=0,[]
        for i in chosen:
            n=len(self.tok.encode(sents[i],add_special_tokens=False))
            if total+n>self.cfg.token_budget: break
            final.append(i); total+=n
        guide=" ".join([sents[i] for i in final])
        docs_join=" ".join(docs)
        # light compression before tokenization to stay safe
        text=f"Summarize the following multiple articles into one concise paragraph:\n{guide}\n{docs_join}"
        text=text[:1500]
        ids=self.tok.encode(text,add_special_tokens=False)
        if len(ids)>self.cfg.max_input_length:
            ids=ids[:self.cfg.max_input_length]
            text=self.tok.decode(ids,skip_special_tokens=True)
        return text

# ---------------- GENERATION ----------------
def safe_batch_generate(model,tok,texts):
    outs=[]
    bs=4; pad_id=tok.pad_token_id or 0
    for i in range(0,len(texts),bs):
        chunk=texts[i:i+bs]
        enc=tok(chunk,max_length=config.max_input_length,truncation=True,
                padding=True,return_tensors="pt").to(config.device)
        with torch.no_grad():
            out=model.generate(**enc,
                num_beams=config.num_beams,
                max_length=config.max_target_length,
                min_length=config.min_length,
                length_penalty=config.length_penalty,
                repetition_penalty=config.repetition_penalty,
                no_repeat_ngram_size=config.no_repeat_ngram_size)
        outs.extend(tok.batch_decode(out,skip_special_tokens=True))
    return [o.strip() for o in outs]

# ---------------- DATASET ----------------
def load_dataset_multi(cfg):
    print("Loading multi-document subset...")
    ds1=load_dataset("cnn_dailymail","3.0.0",split="test[:500]")
    ds2=load_dataset("multi_news",split="test[:500]")
    ds3=load_dataset("xsum",split="test[:500]")
    pool=[]
    for i in range(cfg.total_test_samples):
        pool.append({
            "texts":[ds1[i]["article"],ds2[i]["document"],ds3[i]["document"]],
            "summary":ds2[i]["summary"]
        })
    print(f"✓ Built {len(pool)} multi-document examples")
    return pool

# ---------------- METRICS ----------------
def compute_metrics(refs,cands):
    r=rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"],use_stemmer=True)
    r1,r2,rL=[],[],[]
    for ref,can in zip(refs,cands):
        sc=r.score(ref,can)
        r1.append(sc["rouge1"].fmeasure)
        r2.append(sc["rouge2"].fmeasure)
        rL.append(sc["rougeL"].fmeasure)
    try:
        _,_,F1=bert_score_calculator(cands,refs,lang="en",
                                     model_type="distilbert-base-uncased",
                                     device=config.device)
        b=(F1.cpu().numpy()*100).tolist()
    except:
        b=[0]*len(cands)
    return {"rouge1":r1,"rouge2":r2,"rougeL":rL,"bertscore_f1":b}

# ---------------- MAIN ----------------
def main():
    set_seed(config.seed)
    mkdir(config.output_dir)
    print("="*80)
    print("🚀 HMTAS — Hierarchical Multi-Document Text Abstractive Summarization (FLAN-T5-Base)")
    print("="*80)
    pool=load_dataset_multi(config)
    tok=AutoTokenizer.from_pretrained(config.backbone)
    model=AutoModelForSeq2SeqLM.from_pretrained(config.backbone).to(config.device)
    proc=HMTASProcessor(config,tok)

    print("\nEvaluating HMTAS (guided multi-doc)...")
    t0=time.time()
    guided=[proc.create_guided_input(p["texts"]) for p in pool]
    hmtas=safe_batch_generate(model,tok,guided)
    t_hmtas=time.time()-t0

    print("\nEvaluating Base (flat multi-doc)...")
    t1=time.time()
    flat=[f"Summarize the following news articles into one paragraph:\n{' '.join(p['texts'])[:1500]}" for p in pool]
    base=safe_batch_generate(model,tok,flat)
    t_base=time.time()-t1

    refs=[p["summary"] for p in pool]
    m_h=compute_metrics(refs,hmtas)
    m_b=compute_metrics(refs,base)

    print("\n================= RESULTS =================")
    print("| Metric | HMTAS | Base | Win% |")
    print("|---------|-------|------|------|")
    for k in ["rouge1","rouge2","rougeL","bertscore_f1"]:
        a,b=np.array(m_h[k]),np.array(m_b[k])
        win=(a>b).sum()/len(a)*100
        print(f"| {k:<9}| {np.mean(a)*100:5.2f} | {np.mean(b)*100:5.2f} | {win:4.1f}% |")
    print(f"\nTime(s): HMTAS {t_hmtas:.1f} | Base {t_base:.1f}")
    pd.DataFrame({"ref":refs,"hmtas":hmtas,"base":base}).to_csv(config.output_dir/"summaries.csv",index=False)
    print(f"✓ Saved outputs to {config.output_dir}")

if __name__=="__main__":
    main()


🚀 HMTAS — Hierarchical Multi-Document Text Abstractive Summarization (FLAN-T5-Base)
Loading multi-document subset...


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]


================= RESULTS =================
| Metric | HMTAS | Base | Win% |
|---------|-------|------|------|
| rouge1   | 16.67 | 13.31 | 65.0% |
| rouge2   |  3.96 |  2.08 | 63.0% |
| rougeL   | 10.20 |  8.55 | 64.0% |
| bertscore_f1| 7091.83 | 6856.66 | 68.0% |

Time(s): HMTAS 121.4 | Base 108.3
✓ Saved outputs to hmtas_output
